In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/Users/mr.chen/opt/anaconda3/envs/aipi540/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
csv = pd.read_csv(r'../data/tripadvisor_hotel_reviews.csv')

In [178]:
sentiment_dict = {1: "Negative",
                 2: "Neutral",
                 3: "Positive"}

In [179]:
def rating_to_sentiment(rating):
    if rating>3 and rating<=5:
        return 3
    elif rating == 3:
        return 2
    else:
        return 1

In [180]:
tqdm.pandas()
csv['Sentiment'] = csv['Rating'].progress_apply(rating_to_sentiment)

100%|██████████| 20491/20491 [00:00<00:00, 487426.53it/s]


In [182]:
from torchtext.data.functional import to_map_style_dataset
train_iter = [(label,text)for label,text in zip(csv['Sentiment'].to_list(),csv['Review'].to_list())]
train_dataset = to_map_style_dataset(train_iter)

In [183]:
from torch.utils.data.dataset import random_split
num_train = int(len(train_dataset) * 0.95)
split_train_dataset, split_valid_dataset = random_split(train_dataset, [num_train, len(train_dataset) - num_train])


In [184]:
# Function to tokenize the text
def yield_tokens(data_iter,tokenizer):
    for _, text in data_iter:
        yield tokenizer(text)

# Build vocabulary from tokens of training set
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(yield_tokens(train_iter,tokenizer), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [185]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define collate_batch function to get single collated tensor for batch in form needed by nn.EmbeddingBag
def collate_batch(batch,tokenizer,vocab):
    # Pipelines for processing text and labels
    text_pipeline = lambda x: vocab(tokenizer(x))
    label_pipeline = lambda x: int(x) - 1
    
    label_list, text_list, offsets = [], [], [0]
    # Iterate through batch, processing text and adding text, labels and offsets to lists
    for (label, text) in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)  

In [186]:
batch_size = 32
# Create training, validation and test set DataLoaders using custom collate_batch function
train_dataloader = DataLoader(split_train_dataset, batch_size=batch_size,
                              shuffle=True, collate_fn = lambda x: collate_batch(x,tokenizer,vocab))
val_dataloader = DataLoader(split_valid_dataset, batch_size=batch_size,
                              shuffle=True, collate_fn = lambda x: collate_batch(x,tokenizer,vocab))


# Set up dict for dataloaders to use in training
train_dataloaders = {'train':train_dataloader,'val':val_dataloader}

# Store size of training and validation sets
dataset_sizes = {'train':len(split_train_dataset),'val':len(split_valid_dataset)}

In [221]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(LSTMModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
   
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, num_classes)

        self.init_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets).unsqueeze(1)
        lstm_output, (ht, ct) = self.lstm(embedded)
        #print(embedded.shape, self.embedding(text, offsets).shape,ht.shape,ht[-1].shape,self.fc(ht[-1]).shape)
        return self.fc(ht[-1])

# Model parameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 64
num_classes = 3

# Create the LSTM model, loss function, and optimizer
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    model.train()

    for (labels, text, offsets) in tqdm(train_dataloader):
        text = text.to(device)
        labels = labels.to(device)
        offsets = offsets.to(device)
       # print('text.shape:',text.shape)

        optimizer.zero_grad()
        outputs = model(text, offsets)
        
        loss = criterion(outputs, labels)
   
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()
        

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataset)

    val_loss = 0.0
    val_acc = 0.0
    model.eval()

    val_loss /= len(val_dataloader)
 

    print(f'Epoch {epoch+1}/{num_epochs}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}')


100%|██████████| 609/609 [00:05<00:00, 111.96it/s]


Epoch 1/100, train_loss: 0.6678, train_acc: 0.7165


100%|██████████| 609/609 [00:04<00:00, 122.51it/s]


Epoch 2/100, train_loss: 0.5043, train_acc: 0.7734


100%|██████████| 609/609 [00:04<00:00, 134.37it/s]


...

Epoch 98/100, train_loss: 0.0828, train_acc: 0.9296


100%|██████████| 609/609 [00:04<00:00, 146.01it/s]


Epoch 99/100, train_loss: 0.0416, train_acc: 0.9406


100%|██████████| 609/609 [00:04<00:00, 144.94it/s]

Epoch 100/100, train_loss: 0.0300, train_acc: 0.9440


In [ ]:
import torch
from sklearn.metrics import classification_report

def evaluate(dataloader, model):
    model.eval()
    all_labels, all_predictions = [], []

    with torch.no_grad():
        for idx, (label, text, offset) in enumerate(dataloader):
            predict = model(text, offset)
            predicted_labels = predict.argmax(1)
            all_labels.extend(label.tolist())
            all_predictions.extend(predicted_labels.tolist())

    report = classification_report(all_labels, all_predictions)
    return report
accu_test = evaluate(val_dataloader,model)
print(accu_test)